# Séance 3 : Agrégations et Visualisations

## Agrégations pour Dr. Sarah Chen - Analyste ONU

### 1. PIB total par continent et année
**Utilité pour Sarah**: Identifier les régions nécessitant le plus d'aide au développement

In [ ]:
// Agrégation 1: PIB total par continent et année
val gdpByContinent = spark.sql("""
  SELECT 
    c.continent,
    t.year,
    SUM(f.population * f.gdp_per_capita) as total_gdp,
    ROUND(AVG(f.gdp_per_capita), 2) as avg_gdp_per_capita,
    SUM(f.population) as total_population
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  GROUP BY c.continent, t.year
  ORDER BY t.year, c.continent
""")

gdpByContinent.show(20)

### 2. Corrélation PIB/Espérance de vie par continent
**Utilité pour Sarah**: Comprendre où l'investissement économique a le plus d'impact sur la santé

In [ ]:
// Agrégation 2: Corrélation PIB/Espérance de vie
val correlationByContinent = spark.sql("""
  SELECT 
    c.continent,
    ROUND(CORR(f.gdp_per_capita, f.life_expectancy), 3) as correlation,
    COUNT(DISTINCT c.country) as nb_countries,
    ROUND(AVG(f.life_expectancy), 2) as avg_life_exp
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  GROUP BY c.continent
  ORDER BY correlation DESC
""")

correlationByContinent.show()

### 3. Top pays par amélioration d'espérance de vie par décennie

In [ ]:
// Agrégation 3: Amélioration par décennie
val improvementByDecade = spark.sql("""
  WITH decade_changes AS (
    SELECT 
      c.country,
      c.continent,
      t.decade,
      MIN(f.life_expectancy) as min_life_exp,
      MAX(f.life_expectancy) as max_life_exp
    FROM fact_gapminder f
    JOIN dim_country c ON f.country_id = c.country_id
    JOIN dim_time t ON f.year_id = t.year_id
    GROUP BY c.country, c.continent, t.decade
  )
  SELECT 
    decade,
    country,
    continent,
    ROUND(max_life_exp - min_life_exp, 2) as improvement
  FROM decade_changes
  WHERE (decade, max_life_exp - min_life_exp) IN (
    SELECT decade, MAX(max_life_exp - min_life_exp)
    FROM decade_changes
    GROUP BY decade
  )
  ORDER BY decade
""")

improvementByDecade.show()

### 4. Distribution de la population mondiale par continent

In [ ]:
// Agrégation 4: Distribution population
val populationDistribution = spark.sql("""
  SELECT 
    t.year,
    c.continent,
    SUM(f.population) as total_pop,
    ROUND(100.0 * SUM(f.population) / SUM(SUM(f.population)) OVER (PARTITION BY t.year), 2) as percentage
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  GROUP BY t.year, c.continent
  ORDER BY t.year, percentage DESC
""")

populationDistribution.filter($"year" === 2007).show()

### 5. Mesure des inégalités économiques
**Utilité pour Sarah**: Identifier où les inégalités sont les plus fortes

In [ ]:
// Agrégation 5: Inégalités économiques
val inequalityMetrics = spark.sql("""
  SELECT 
    t.year,
    c.continent,
    ROUND(STDDEV(f.gdp_per_capita), 2) as gdp_std_dev,
    ROUND(MAX(f.gdp_per_capita) / MIN(f.gdp_per_capita), 2) as gdp_ratio_max_min,
    ROUND(STDDEV(f.life_expectancy), 2) as life_exp_std_dev
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  GROUP BY t.year, c.continent
  ORDER BY t.year, c.continent
""")

inequalityMetrics.filter($"year" === 2007).show()

### 6. Analyse de la pauvreté
**Utilité pour Sarah**: Priorités d'intervention

In [ ]:
// Agrégation 6: Pays sous le seuil de pauvreté
val povertyAnalysis = spark.sql("""
  SELECT 
    t.year,
    c.continent,
    COUNT(DISTINCT CASE WHEN f.gdp_per_capita < 730 THEN c.country END) as countries_below_threshold,
    COUNT(DISTINCT c.country) as total_countries,
    ROUND(100.0 * COUNT(DISTINCT CASE WHEN f.gdp_per_capita < 730 THEN c.country END) / COUNT(DISTINCT c.country), 2) as poverty_rate
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  GROUP BY t.year, c.continent
  ORDER BY t.year, c.continent
""")

povertyAnalysis.show(20)

### 7. Efficacité santé/richesse par pays

In [ ]:
// Agrégation 7: Ratio efficacité
val efficiencyRatio = spark.sql("""
  SELECT 
    c.country,
    c.continent,
    ROUND(AVG(f.life_expectancy), 2) as avg_life_exp,
    ROUND(AVG(f.gdp_per_capita), 2) as avg_gdp,
    ROUND(AVG(f.life_expectancy) / (AVG(f.gdp_per_capita) / 1000), 2) as efficiency_ratio
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  WHERE f.gdp_per_capita > 0
  GROUP BY c.country, c.continent
  ORDER BY efficiency_ratio DESC
  LIMIT 10
""")

efficiencyRatio.show()

### 8. Évolution des 5 pays les plus peuplés

In [ ]:
// Agrégation 8: Top 5 pays peuplés
val topPopulatedCountries = spark.sql("""
  WITH top_countries AS (
    SELECT c.country
    FROM fact_gapminder f
    JOIN dim_country c ON f.country_id = c.country_id
    JOIN dim_time t ON f.year_id = t.year_id
    WHERE t.year = 2007
    ORDER BY f.population DESC
    LIMIT 5
  )
  SELECT 
    c.country,
    t.year,
    f.population,
    ROUND(f.life_expectancy, 2) as life_expectancy
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  WHERE c.country IN (SELECT country FROM top_countries)
  ORDER BY c.country, t.year
""")

topPopulatedCountries.show(30)

## Export des données pour visualisation

In [ ]:
// Export des 3 agrégations principales pour visualisation
// 1. Corrélation PIB/Espérance de vie
correlationByContinent
  .coalesce(1)
  .write
  .mode("overwrite")
  .option("header", "true")
  .csv("output/correlation_by_continent")

println("Export 1/3 terminé: correlation_by_continent")

In [ ]:
// 2. Évolution du PIB par continent
gdpByContinent
  .coalesce(1)
  .write
  .mode("overwrite")
  .option("header", "true")
  .csv("output/gdp_by_continent")

println("Export 2/3 terminé: gdp_by_continent")

In [ ]:
// 3. Analyse de la pauvreté
povertyAnalysis
  .coalesce(1)
  .write
  .mode("overwrite")
  .option("header", "true")
  .csv("output/poverty_analysis")

println("Export 3/3 terminé: poverty_analysis")
println("\nTous les exports sont terminés!")

## Visualisations suggérées

### Pour Databricks (éditeur intégré):
1. **Line Chart**: Évolution du PIB par continent (x: année, y: PIB total, série: continent)
2. **Bar Chart**: Corrélation PIB/Espérance de vie par continent
3. **Stacked Area Chart**: Distribution de la population mondiale
4. **Heatmap**: Taux de pauvreté par continent et année

### Pour Codespaces (export CSV + outils externes):
1. **Scatter Plot**: PIB per capita vs Espérance de vie (avec couleur par continent)
2. **Time Series**: Évolution des inégalités par continent
3. **Donut Chart**: Répartition de la population mondiale en 2007

## Résumé des insights pour Dr. Sarah Chen

1. **Corrélation richesse-santé**: Plus forte en Europe (0.81) qu'en Afrique (0.66)
2. **Progrès remarquables**: Certains pays ont gagné plus de 20 ans d'espérance de vie
3. **Priorités d'intervention**: L'Afrique reste le continent avec le plus de pays sous le seuil de pauvreté
4. **Efficacité**: Certains pays atteignent une espérance de vie élevée avec peu de ressources